This notebook uses patient 1 artery centerlines and Gjerge Splitting paper technique.
Copied from VesselSingularity

## Import packages

In [ ]:
import numpy as np 
from fenics import * 
from dolfin import * 
from mpi4py import MPI
from numpy import linalg as la

In [ ]:
#imports for distance calculation
from numpy import array
from numpy.linalg import norm
from numpy import dot

In [ ]:
#import for json files
import vtk
from vtk.util import numpy_support as VN
import json
import pandas as pd

In [ ]:
import os
import time

## Create Mesh that extends the whole length of the artery of patient 1

In [ ]:
mesh_liver = BoxMesh(Point(105.339, 106.092, -10), Point(231.518, 180.395, 213), 30, 30, 30)

#Bounds: 
    #x: 105.339 to 231.518
    #y: 06.092 to 180.395
    #z: 0.915998 to 202.829

## Define distance to point x from a line formed by point a and b 

In [ ]:
def distance(x, a, b):
    d = np.subtract(b, a)/norm(np.subtract(b,a))
    v = Expression(('x[0]-a0', 'x[1]-a1', 'x[2]-a2'), degree=1, a0=a[0], a1=a[1], a2=a[2])
    t = Expression('v[0]*d0+v[1]*d1+v[2]*d2', degree=1, v=v, d0=d[0], d1=d[1], d2=d[2])
    p = Expression(('a0+t*d0', 'a1+t*d1', 'a2+t*d2'), degree=1, a0=a[0], a1=a[1], a2=a[2], d0=d[0], d1=d[1], d2=d[2], t=t)
    rad = Expression('sqrt(pow((p[0]-x[0]), 2) + pow((p[1]-x[1]), 2) + pow((p[2]-x[2]), 2))', degree=1, p=p)
    return rad

## Writing a new function for equation 5.13 and 5.14 

In [ ]:
def branch_vessel(mesh, folder, name_corr, name_all):
    st = time.time() 
    
    #Define Function Space
    V = FunctionSpace(mesh, "CG", 1)
    
    # Define Helper functions 
    x = SpatialCoordinate(mesh)
    w_all = Function(V)
    u_all = Function(V)
    
    #read in the radius vzlues 
    radius_data=pd.read_csv(folder + 'radius.tsv', sep='\t')
    DIR = folder + "centerlines/"
    line_count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
    
    #calculate points_all
    point_sets = 0
    for i in range(line_count):
        f = open(DIR + 'Centerline_' + str(i) + '.mrk.json', "r")
        data = json.loads(f.read())
        points = [y['position'] for y in np.concatenate([x['controlPoints'] for x in data['markups']]).flat]        
        
        point_sets += (len(points)-1)
        print(str(point_sets) + " is the number of total points after centerline "  + str(i))
    
    #run for each branch
    for i in range(line_count):
        
        # load in the file for each branch
        print("centerline " + str(i))
        f = open(DIR + 'Centerline_' + str(i) + '.mrk.json', "r")
        data = json.loads(f.read())
        points = [y['position'] for y in np.concatenate([x['controlPoints'] for x in data['markups']]).flat]
        
        # load in radius for each 
        E = radius_data['Radius'][i]
    
        for j in range(len(points)-1): #iterate over each point to get E*G
                            
            # starting and ending point for each straight line in branch
            a = points[j]
            b = points[j+1]
            
            # coefficients for G calculation
            diff = np.subtract(b, a)
            L = norm(diff)
            tau = diff/L
            tol = 1E-14

            # Define radius -- the project of point onto line 
            rad = distance(x, a, b)

            #Define G variable ln(r)
            rb = Expression('sqrt( pow(x[0]-b0,2) + pow(x[1]-b1,2) + pow(x[2]-b2,2))', degree=1, b0=b[0], b1=b[1], b2=b[2])
            ra = Expression('sqrt( pow(x[0]-a0,2) + pow(x[1]-a1,2) + pow(x[2]-a2,2))', degree=1, a0=a[0], a1=a[1], a2=a[2])
            tauax = Expression('tau0*(a0-x[0]) + tau1*(a1-x[1]) + tau2*(a2-x[2])', degree=1,  tau0=tau[0], a0=a[0], tau1=tau[1], a1=a[1], tau2=tau[2], a2=a[2])
            G = Expression('std::log((rb + L + tauax)/(ra + tauax + eps))', degree=0, rb=rb, ra=ra, L=L, a2=a[2], tauax=tauax, eps=epsval)
            dG = Expression(('(((x[0]-b0)/rb)-tau0)/(rb + L + tauax + eps) - ((((x[0]-a0)/ra)-tau0)/(ra + tauax + eps))', '(((x[1]-b1)/rb)-tau1)/(rb + L + tauax + eps) - ((((x[1]-a1)/ra)-tau1)/(ra + tauax + eps))', '(((x[2]-b2)/rb)-tau2)/(rb + L + tauax + eps) - ((((x[2]-a2)/ra)-tau2)/(ra + tauax + eps))'), degree=1, rb=rb, ra=ra, L=L, a0=a[0], b0=b[0], a1=a[1], b1=b[1], a2=a[2], b2=b[2], tauax=tauax, tau0=tau[0], tau1=tau[1], tau2=tau[2], eps=epsval) 

            #EfG = Expression('E*G+EG_total', degree=1, E=E, G=G, EG_total=EfG)
            
            # Define boundary condition 
            uD = Constant(1.0)
            wD = Expression('4*pi*(1/point_sets)*uD - E*G', degree=1, E=E, G=G, uD=uD, point_sets=point_sets)

            def boundary(x, on_boundary):
                return on_boundary

            bc = DirichletBC(V, wD, boundary)

            # Define variational problem
            wh = TrialFunction(V)
            v = TestFunction(V)
            f = Constant(0.0)
            a = dot(grad(wh), grad(v))*dx
            L = f*v*dx

            wh = Function(V)
            solve(a == L, wh, bc)

            w_h = interpolate(wh, V)
            w_all.vector()[:] += w_h.vector()[:]
            
            u = Expression('E*G', degree=1, E=E, G=G)
            u_h = interpolate(u, V)
            u_all.vector()[:] += u_h.vector()[:]
        
    # save the correction term    
    et = time.time()
    lapsed_time = et-st
    print(str(lapsed_time))
    
    print("wh is presaved")
    vtkfile = File(name_cor)
    print("wh is saved")
    vtkfile << w_all
    save_w = time.time()
    print("saved w after " + str(save_w-et))
    
    u_h = Expression('((1.0)/(4*pi))*(u_all + w_all)', degree=1, u_all=u_all, w_all=w_all)
    u_all = interpolate(u_h, V)
    created_u_time = time.time()

    # save the total u term
    print("uh is presaved")
    vtkfile = File(name_all)
    print("uh is saved")
    vtkfile << u_all
    
    save_u = time.time()
    print("saved u after " + str(save_u - created_u_time))



## Import JSON Files and run FEM

Centerlines are created from the given starting and ending point. The json files are for each individual branch with the branches ending point being equal to the starting point of the next branch

mesh = the mesh that encapsulates the whole liver 

folder = where the centerline information is held, specifically the radius 

name_cor = location of the output of the correction term

name_all = location of the output of the full term 

In [ ]:
mesh = mesh_liver
folder = "/Users/bilyana/Documents/vessel_segmentation/ircad_data/patient_1/all_branches/"
name_cor = folder + "test.pvd"
name_all = folder + "test.pvd"
amount = 1

branch_vessel(mesh, folder, name_cor, name_all)

In [ ]:
mesh = mesh_liver_2
folder = "/Users/bilyana/Documents/vessel_segmentation/ircad_data/patient_1/all_branches/"
name_cor = folder + "corr_1.pvd"
name_all = folder + "all_1.pvd"
amount = 1

branch_vessel(mesh, folder, name_cor, name_all, amount)